# Homework 3 | Derek Hu | COEN 140

In [1]:
import numpy as np
import math

In [2]:
dataTrain = np.loadtxt('crime-train.txt', delimiter = '\t', skiprows = 1)
dataTest = np.loadtxt('crime-test.txt', delimiter = '\t', skiprows = 1)

xTrain = dataTrain[:, 1:]
# Append 1s to the end of xTrain
ones = np.ones(len(xTrain))
xTrain = np.column_stack((xTrain, ones))
yTrain = dataTrain[:, 0:1]

xTest = dataTest[:, 1:]
# Append 1s to the end of xTest
ones = np.ones(len(xTest))
xTest = np.column_stack((xTest, ones))
yTest = dataTest[:, 0:1]

## Linear Regression w/ Closed Form Solution

In [3]:
# Perform linear regression and return prediction for "test" data
def linear_regression(x, y, test):
    left = np.linalg.inv(np.dot(x.T, x))
    right = np.dot(x.T, y)
    w = np.dot(left, right)
    
    prediction = []
    
    for i in test:
        prediction.append(np.dot(w.T, i))
        
    return prediction

In [4]:
# Compute RMSE from predicted y and actual y
def get_RMSE(prediction, y):
    n = len(prediction)
    diff = prediction - y
    Sum = 0

    for i in diff:
        Sum += (i ** 2)

    RMSE = math.sqrt(Sum / n)
    
    return RMSE

In [5]:
# Make predictions and calculate RMSE for training data
train_prediction = linear_regression(xTrain, yTrain, xTrain)
train_RMSE = get_RMSE(train_prediction, yTrain)
print("Training Linear RMSE is ", train_RMSE)

# Make predictions and calculate RMSE for test data
test_prediction = linear_regression(xTrain, yTrain, xTest)
test_RMSE = get_RMSE(test_prediction, yTest)
print("Testing Linear RMSE is ", test_RMSE)

Training Linear RMSE is  0.12768967421762198
Testing Linear RMSE is  0.1458346449094983


## Ridge Regression w/ Closed Form Solution & K-Fold Cross Validation

In [6]:
# Perform ridge regression and return prediction for "test" data
def ridge_regression(x, y, test, Lambda):
    left = np.linalg.inv(np.dot(x.T, x) + (Lambda * np.identity(len(x.T))))
    right = np.dot(x.T, y)
    w = np.dot(left, right)
    
    prediction = []
    
    for i in test:
        prediction.append(np.dot(w.T, i))
        
    return prediction

In [7]:
Lambda = 400
RMSEs = np.zeros((10,5))
Min = 0

# Do k-fold cross validation with k = 5
for i in range(10):
    for k in range(5):
        xTrain_k = np.concatenate((xTrain[0:int(k*(len(xTrain)/5))], xTrain[int((k+1)*(len(xTrain)/5)):int(len(xTrain))]))
        yTrain_k = np.concatenate((yTrain[0:int(k*(len(yTrain)/5))], yTrain[int((k+1)*(len(yTrain)/5)):int(len(yTrain))]))
        
        xTest_k = xTrain[int(k*(len(xTrain)/5)):int((k+1)*(len(xTrain)/5))]
        yTest_k = yTrain[int(k*(len(yTrain)/5)):int((k+1)*(len(yTrain)/5))]

        prediction = ridge_regression(xTrain_k, yTrain_k, xTest_k, Lambda)

        RMSEs[i, k] = get_RMSE(prediction, yTest_k)
    Lambda /= 2

# Get average RMSE for each lambda and choose optimal lambda
for i in range(9):
    if (sum(RMSEs[i+1]) / 5) < (sum(RMSEs[Min]) / 5):
        Min = i + 1

for j in range(10):
    print("Lambda", (400 / (2 **j)), ":", sum(RMSEs[j]) / 5)

optLambda = 400 / (2 ** Min)

Lambda 400.0 : 0.149512776191
Lambda 200.0 : 0.140694018453
Lambda 100.0 : 0.137276772232
Lambda 50.0 : 0.136155943928
Lambda 25.0 : 0.135915859199
Lambda 12.5 : 0.136022374462
Lambda 6.25 : 0.136267864941
Lambda 3.125 : 0.136570956936
Lambda 1.5625 : 0.136887640732
Lambda 0.78125 : 0.137176268069


In [8]:
# Make predictions and calculate RMSE for test data
testR_prediction = ridge_regression(xTrain, yTrain, xTest, optLambda)
testR_RMSE = get_RMSE(testR_prediction, yTest)
print("Testing Ridge RMSE is ", testR_RMSE)

Testing Ridge RMSE is  0.14574650707057937


## Linear Regression w/ Gradient Descent

In [9]:
# Compute loss function for linear regression
def linear_loss_function(x, y, w):
    left = (y - np.dot(x, w)).T
    right = (y - np.dot(x, w))
    L = np.dot(left, right)
    
    return L

alpha = .00001

In [10]:
# Use gradient descent until convergence to obtain w
w0 = np.random.normal(0, 1, (xTrain.shape[1],1))
L1 = 1
L0 = 0

while (abs(L1 - L0) > .0000001):
    w1 = w0 - (alpha * np.dot(xTrain.T, (np.dot(xTrain, w0) - yTrain)))
    L0 = linear_loss_function(xTrain, yTrain, w0)
    L1 = linear_loss_function(xTrain, yTrain, w1)
    w0 = w1

In [11]:
prediction = []

# Make predictions and calculate RMSE for training data
for i in xTrain:
    prediction.append(np.dot(w0.T, i))

trainLGD_RSME = get_RMSE(prediction, yTrain)
print("Training LGD RMSE is ", trainLGD_RSME)

# Make predictions and calculate RMSE for test data
for i in xTest:
    prediction.append(np.dot(w0.T, i))

testLGD_RMSE = get_RMSE(test_prediction, yTest)
print("Testing LGD RMSE is ", testLGD_RMSE)

Training LGD RMSE is  0.12769916223589392
Testing LGD RMSE is  0.1458346449094983


## Ridge Regression w/ Gradient Descent

In [12]:
# Compute loss function for ridge regression
def ridge_loss_function(x, y, w):
    left = (y - np.dot(x, w)).T
    right = (y - np.dot(x, w))
    L = np.dot(left, right) + (optLambda * np.dot(w0.T, w0))
    
    return L

alpha = .00001

In [13]:
# Use gradient descent until convergence to obtain w
w0 = np.random.normal(0, 1, (xTrain.shape[1],1))
L1 = 1
L0 = 0

while (abs(L1 - L0) > .0000001):
    w1 = w0 - (alpha * (np.dot(xTrain.T, (np.dot(xTrain, w0) - yTrain)) + (optLambda * w0)))
    L0 = ridge_loss_function(xTrain, yTrain, w0)
    L1 = ridge_loss_function(xTrain, yTrain, w1)
    w0 = w1

In [14]:
prediction = []

# Make predictions and calculate RMSE for test data
for i in xTest:
    prediction.append(np.dot(w0.T, i))

testRGD_RMSE = get_RMSE(prediction, yTest)
print("Testing RGD RMSE is ", testRGD_RMSE)

Testing RGD RMSE is  0.145746402627983
